In [2]:
import datetime as dt
d1 = dt.datetime(2022,4,1)
d2 = dt.datetime(2022,4,1,tzinfo=dt.timezone.utc)

d1.utcoffset()
# d3 = dt.datetime(d1, tzinfo=dt.timezone.utc)
print(d1) 
print(d2)
print(d1.timestamp())
print(d2.timestamp())
print(d1.tzname()) 
print(d2.tzname())
if d1.tzinfo == None:
    d1 = d1.replace(tzinfo= dt.timezone.utc)
    print(1)
if d1.tzinfo == None:
    d1 = d1.replace(tzinfo= dt.timezone.utc)
    print(2)





2022-04-01 00:00:00
2022-04-01 00:00:00+00:00
1648760400.0
1648771200.0
None
UTC
1


In [7]:
"""Запуск анализа новыйх файлов XML в заданной папке"""
import os

# work_dir = 'D:\\YandexDisk\\Мои ДОКУМЕНТЫ\\ООО Арсенал-строй (уу)\\!!! Управленческй учет\\Закрытие 2022-04 (апрель)\\'

import modules.pe_methods as pe
file_name_input = 'D:\\YandexDisk\\Мои ДОКУМЕНТЫ\\ООО Арсенал-строй (уу)\\!!! Управленческй учет\\Закрытие 2022-03 (март)\\\
Выписка по дебетовой карте (на русском) Platinum 3897.xml'

pe.convert_payments_to_json(file_name_input, 'D:\\tmp\\ttt.json')


'Ok'

In [3]:
import modules.pe_methods as pe
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import time
import datetime as dt
import tkinter as tk
import tkinter.filedialog as fd
import tkinter.messagebox as mb
import modules.my_methods as mm
import modules.win_messages as win
from  modules.models import *
import os, datetime, json

# file_name_input = 'D:\\YandexDisk\\Мои ДОКУМЕНТЫ\\ООО Арсенал-строй (уу)\\!!! Управленческй учет\\Закрытие 2022-04 (апрель)\\\
file_name_input = 'D:\\YandexDisk\\Мои ДОКУМЕНТЫ\\ООО Арсенал-строй (уу)\\!!! Управленческй учет\\Закрытие 2022-04 (апрель)\\\
Выписка по дебетовой карте (на русском) MIR •••• 6753.xml'
# Выписка по дебетовой карте (на русском) Platinum.xml'
# Выписка по дебетовой карте (на русском) Maestro •••• 6632.xml'

# res = pe.load_payments_from_XML_sber(file_name_input)
# res

In [4]:
def load_payments_from_XML_sber(file_name):
    # возвращает словарь результатов:
    # df: DataFrame
    # Параметры выписки (d1, d2, sum1, sum2, приходы, расходы, метка карты)

    res = {}
    res["error_text"] = "" # значение поумолчанию, т.е. нет ошибок
    res['success'] = False
    res['prop'] = [0,0,0,0]  # res['prop'] [Остаток1, Остаток2, Расходы, Приходы]

    # tag_root
    xlm = ET.parse(file_name)
    tag_root = xlm.getroot()

    level =   '{urn:schemas-microsoft-com:office:spreadsheet}Worksheet'
    level += '/{urn:schemas-microsoft-com:office:spreadsheet}Table'
    level += '/{urn:schemas-microsoft-com:office:spreadsheet}Row'

    ind = 0
    rows = []
    rows1 = []

    # считать данные в список
    for tag_row in tag_root.findall(level):
        row = []
        for i_cel, tag in enumerate(tag_row.findall('{urn:schemas-microsoft-com:office:spreadsheet}Cell')):            
            text = ''
            for t in tag.itertext():
                text = t
            row.append(text)
        if ind < 19:
            rows.append(row)
        else:
            if len(row)==8:
                rows1.append(row) 
        ind +=1

    
    if len(rows) < 19:
        res['error_text'] = f'При попытке чтения XML файла возникла ошибка формата файла. Недостаточная длина строк (код 201).\n\
            количество строк: {len(rows)}'
        res['success'] = False
        return res

    # Заголовки
    # Найти метку
    mark_text = 'Карта'
    if rows[5][0] != mark_text:
        res['error_text'] = 'Ошибка формата файла. Не найдена метка (код 202)'
        res['success'] = False
        return res
        
    # Найти метку карты (в той же строке)
    res['card_mark'] = rows[6][0]

    # 'd1', 'd2', 'period'
    text = rows[7][0]
    res['period'] = text    # Итого по операциям с 01.03.2022 по 31.03.2022
    i1 = text.find(' с ') + 3
    i2 = text.find(' по ') + 4
    if len(text) == 45: # len(text)=45
        t1 = text[21:31]
        t2 = text[35:45]
        res['d1'] = datetime.datetime.strptime(t1, '%d.%m.%Y')
        res['d2'] = datetime.datetime.strptime(t2, '%d.%m.%Y')# + datetime.timedelta(days=1)
    else:
        error_text = f'При попытке чтения XML \n\
            {file_name} \n не считаны данные о периоде выписки. \n\
            длина строки д.б. равна 45: ltn(text)={len(text)}'
        res["error_text"] = error_text
        res['success'] = False
        return res


    # res['prop'] [Остаток1, Остаток2, Расходы, Приходы]
    res['prop'][0] = mm.text_to_float(rows[9][0])
    res['prop'][1] = mm.text_to_float(rows[9][2])
    res['prop'][2] = mm.text_to_float(rows[9][4])
    res['prop'][3] = mm.text_to_float(rows[9][6])


    # Обрезать строки заголовка
    rows = rows1

    # Обработать rows в массив записей rows_ar
    len1 = len(rows)
    len2 = len1//2
    if len1%2 != 0:
        res["error_text"] = f'При попытке чтения XML файла считано не четное количество строк в таблице: {len1}'
        res['success'] = False
        return res
    rows2 = []

    #   Date	    Time  ...   Name	...	         Amount	....	Balance	Amount_num	Balance_num
    # 0	2022-03-31	15:44		Перевод с карты		5 000,00		337.94	5000.00	337.94
    # 1	2022-03-31	202880		SBOL перевод 2202****6497 В. СЕРГЕЙ СЕРГЕЕВИЧ					NaN	NaN
    for i2 in range(0, len2):
        i1 = 2*i2
        r1 = rows[i1]
        r2 = rows[i1+1]    
        row = []
        
        # date = DateTimeField()
        val = dt.datetime.strptime(r1[0][0:10] + ' ' + r1[1], '%Y-%m-%d %H:%M')
        row.append(val)

        # date_account = DateTimeField(null=True)    
        val = dt.datetime.strptime(r2[0][0:10], '%Y-%m-%d')
        row.append(val)

        # category = CharField(null=True) 
        val = r1[3]
        row.append(val)

        # name = CharField(null=True)
        val = r2[3]
        row.append(val)

        # amount = FloatField()
        st = r1[5]
        val = mm.text_to_float(st) if st[0] == '+' else -mm.text_to_float(st)
        row.append(val)

        # balance
        val = mm.text_to_float(r1[7])
        row.append(val)

        # Сохранить в массив 
        rows2.append(row)

    # res['df']
    # Создать DataFrame из списка rows
    columns=["date", "date_account", "category", "name", "amount", "balance"]
    df = pd.DataFrame(columns=columns, data=rows2).sort_values(by='date')
    res['df'] = df
    res['mess'] = f"Записать/обновить банковскую выписку? \n\n\
            Карта: {res['card_mark']}\n\
            {res['period']}\n\
            Приходы: {res['prop'][3]:_.2f}  \n\
            Расходы: {res['prop'][2]:_.2f}  \n\
            Остаток на начло периода: { res['prop'][0]:_.2f}\n\
            Остаток на конец периода: { res['prop'][1]:_.2f}".replace('_', ' ')
    res['success'] = True
    # df.to_csv('test_out2.csv', sep=';', decimal=',', encoding='cp1251')

    return res

res = load_payments_from_XML_sber(file_name_input)
res


{'error_text': '',
 'success': True,
 'prop': [153.52, 9704.02, 47449.5, 57000.0],
 'card_mark': 'MIR •••• 6753',
 'period': 'Итого по операциям с 01.04.2022 по 30.04.2022',
 'd1': datetime.datetime(2022, 4, 1, 0, 0),
 'd2': datetime.datetime(2022, 4, 30, 0, 0),
 'df':                   date date_account                               category  \
 22 2022-04-05 19:52:00   2022-04-06  Комунальные платежи, связь, интернет.   
 21 2022-04-06 12:18:00   2022-04-06                        Прочие операции   
 20 2022-04-06 12:19:00   2022-04-06                        Перевод с карты   
 19 2022-04-08 07:53:00   2022-04-09  Комунальные платежи, связь, интернет.   
 18 2022-04-08 09:32:00   2022-04-09                           Все для дома   
 17 2022-04-08 15:31:00   2022-04-09                           Супермаркеты   
 16 2022-04-09 20:31:00   2022-04-09                        Перевод с карты   
 15 2022-04-10 06:55:00   2022-04-11                        Прочие операции   
 14 2022-04-10 14:52

In [2]:
file_name_input

NameError: name 'file_name_input' is not defined

In [1]:

def load_payments_from_XML_RNKB(file_name):
    # возвращает словарь результатов:
    # df: DataFrame
    # Параметры выписки (d1, d2, sum1, sum2, приходы, расходы, метка карты)
    # res['success'] = True - флаг успешного чтения формата файла

    res = {}
    res["error_text"] = "" # значение поумолчанию, т.е. нет ошибок
    res['success'] = False
    res['prop'] = [None, None,0,0]  # res['prop'] [Остаток1=None, Остаток2=None, Расходы, Приходы]


    # tag_root
    xlm = ET.parse(file_name)
    tag_root = xlm.getroot()

    level =   '{urn:schemas-microsoft-com:office:spreadsheet}Worksheet'
    level += '/{urn:schemas-microsoft-com:office:spreadsheet}Table'
    level += '/{urn:schemas-microsoft-com:office:spreadsheet}Row'

    rows = []#{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[]}

    # считать данные в список
    for tag_row in tag_root.findall(level):        
        row = []
        for i_cel, tag in enumerate(tag_row.findall('{urn:schemas-microsoft-com:office:spreadsheet}Cell')):            
            text = ''
            for t in tag.itertext():
                text = t
            row.append(text)
        rows.append(row)
    
    if len(rows) < 11:
        res['error_text'] = 'При попытке чтения XML файла возникла ошибка формата файла. Недостаточная длина строк (код 101)'
        res['success'] = False
        return res

    # Заголовки
    # Найти метку
    mark_text = 'Карта'
    if rows[1][0] != mark_text:
        res['error_text'] = 'Ошибка формата файла. Не найдена метка (код 102)'
        res['success'] = False
        return res

    # Найти метку карты (в той же строке)
    res['card_mark'] = rows[1][1]

    # 'd1', 'd2', 'period'
    d1 = datetime.datetime.strptime(rows[3][1], '%d.%m.%Y')
    d2 = datetime.datetime.strptime(rows[4][1], '%d.%m.%Y')
    res['d1'] = d1
    res['d2'] = d2# + datetime.timedelta(days=1)
    res['period'] = f'за период с {d1.strftime("%d.%m.%Y")} по {d2.strftime("%d.%m.%Y")}'

    # res['prop'] [Остаток1=None, Остаток2=None, Расходы, Приходы]
    # "Поступления"
    res['prop'][3] = mm.text_to_float(rows[5][1])
    # "Расходы"
    res['prop'][2] = mm.text_to_float(rows[6][1])

# row  11 ----------------- <Element '{urn:schemas-microsoft-com:office:spreadsheet}Row' at 0x000001B75BE28C20>
# [0] 01.04.2022 08:39
# [1] 01.04.2022 00:00
# [2] 
# [3] -50 000.00
# [4] 0.00
# [5] -50 000.00
# [6] RUR
# [7] Другое
# [8]
# row  12 ----------------- <Element '{urn:schemas-microsoft-com:office:spreadsheet}Row' at 0x000001B75BE2F270>
# [0] 01.04.2022 14:37
# [1] 01.04.2022 00:00
# [2] Выдача наличных RUS SEVASTOPOL ATM-8316
# [3] -49 000.00
# [4] 0.00
# [5] -49 000.00
# [6] RUR
# [7] Снятие наличных
# [8]

    rows2 = []
    n1 = len(rows)
    for ind, r1 in enumerate(rows[11:n1]):
        row = []
        
        # date = DateTimeField()
        val = dt.datetime.strptime(r1[0], '%d.%m.%Y %H:%M')
        row.append(val)

        # date_account = DateTimeField(null=True)    
        val = dt.datetime.strptime(r1[1], '%d.%m.%Y %H:%M')
        row.append(val)

        # category = CharField(null=True) 
        val = r1[7]
        row.append(val)

        # name = CharField(null=True)
        val = r1[2]
        row.append(val)

        # amount = FloatField()
        st = r1[3]
        val = mm.text_to_float(st)
        row.append(val)

        # balance
        val = None
        row.append(val)

        # Сохранить строку
        rows2.append(row)

    # res['df']
    # Создать DataFrame из списка rows
    columns=["date", "date_account", "category", "name", "amount", "balance"]
    df = pd.DataFrame(columns=columns, data=rows2).sort_values(by='date')
    res['df'] = df
    res['mess'] = f"Записать/обновить банковскую выписку? \n\n\
            Карта: {res['card_mark']}\n\
            {res['period']}\n\
            Приходы: {res['prop'][3]:_.2f}  \n\
            Расходы: {res['prop'][2]:_.2f}".replace('_', ' ')
    res['success'] = True
    # df.to_csv('test_out2.csv', sep=';', decimal=',', encoding='cp1251')

    return res


